In [11]:
import cv2
import re
import os
import numpy as np
from statistics import median

In [12]:
nImages = None
imageSetNumber = 4
kFirstMatches = 2

nKeyPointsInitial = 15000
goodMatchRatio = 0.7

histogramEqualizationWIndowSize = (2,2)
xParameter=2.0

BlurTuple = (5,5)

resizeFactor = 5
resizeTuple = (4160//resizeFactor, 2340//resizeFactor)



In [13]:
imageSetPath = "./In-Sample/"+str(imageSetNumber)+"/"


images = [ cv2.imread((imageSetPath+image)) for image in os.listdir(imageSetPath) if image[-3:]=="jpg"]
nImages = len(images)
# for i in range(len(images)) :
#     images[i] = cv2.cvtColor(images[i], cv2.COLOR_RGB2GRAY)

# print(len(images))



for i in range(len(images)):
    blur = cv2.blur(images[i], BlurTuple)
    images[i] = cv2.resize(blur,resizeTuple)
    


In [14]:
def getMatches(img1, img2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = False)
    orb = cv2.ORB_create(nfeatures=nKeyPointsInitial)

    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
    
    matches = bf.knnMatch(descriptors1, descriptors2, k=kFirstMatches)
    matches = selectGoodMatches(matches)
    
#     print("matches list is:", matches)
    
    return matches, keypoints1, keypoints2

In [15]:
def getOrdering(images):
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = False)
    orb = cv2.ORB_create(nfeatures=nKeyPointsInitial)
    splitImages = []
    for image in images:
        cols = len(image[0])
        splitImages.append(image[:,0:(cols//2),:])
        splitImages.append(image[:,(cols//2):cols,:])
    
    images = splitImages
    nImages=len(images)
    
    
    maxIndices = None, None
    maxMatches = []
    

    matchDictionary = {}
    for i in range(nImages):
        matchDictionary[i] = {}
    
    
    keypoints = []
    descriptors = []
    
    for image in images :
        keypoints_orb, descriptors_orb = orb.detectAndCompute(image, None)
        keypoints.append(keypoints_orb)
        descriptors.append(descriptors_orb)
    
    
    
    for i in range(nImages):
        print("\n")
        for j in range(i+1,nImages):
            matches = bf.knnMatch(descriptors[i], descriptors[j], k=kFirstMatches)
            matches = selectGoodMatches(matches)
            matchDictionary[i][j] = matches
            matchDictionary[j][i] = matches
            print(i,j,len(matchDictionary[i][j]))
        
            
    
    
    imageClusters = [[2*i, 2*i+1] for i in range(nImages//2)]
    
    for n in range(len(imageClusters),1,-1):
        
        closestClusters = None, None
        maxMatch = 0
        
        print(imageClusters)
        for i in range(len(imageClusters)):
            img1 = imageClusters[i][-1]
            for j in range(len(imageClusters)):
                if(i!=j):
                    img2 = imageClusters[j][0]
                    if(len(matchDictionary[img1][img2])>maxMatch):
                        maxMatch = len(matchDictionary[img1][img2])
                        closestClusters = i, j
#         print(closestClusters)
        ind1, ind2 = closestClusters
        imageClusters[ind1]+=imageClusters[ind2]
        del imageClusters[ind2]
    
    returnIndices = [imageClusters[0][i]//2 for i in range(0,len(imageClusters[0]),2)]
    print(returnIndices)
    return returnIndices, matchDictionary

In [16]:
def selectGoodMatches(matches):
    # Apply ratio test
    good = []
    for m in matches:
        if m[0].distance < goodMatchRatio*m[1].distance:
            good.append(m[0])
    matches = np.asarray(good)
    return matches

In [7]:
def rgbEqualize(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
#     img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=histogramEqualizationWIndowSize)
    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img_output

In [8]:
def warpTwoImages(img1, img2):
    '''warp img2 to img1 with homograph H'''
    
    matches, keypoints1, keypoints2 = getMatches(img1, img2)
    
    print("shape of matches is:", matches.shape)
    
    src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC,xParameter)
    
    
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    transformedPoints = pts2_.reshape(4,2)
#     print(pts2_.reshape(4,2))
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
    
    
    translatedPoints = [[point[0]+t[0],point[1]+t[1]] for point in transformedPoints]
#     for point in transformedPoints:
#         x,y = point
#         temparray = [x+t[0], y+t[1]]
#         translatedPoints.append(temparray)

    
    warpedImage = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    
    
    mask = np.zeros(warpedImage.shape, dtype=np.uint8)
    cv2.fillPoly(mask, np.array([translatedPoints], dtype=np.int32), (255, 255, 255))
    
    
    result = np.zeros(warpedImage.shape, dtype=np.uint8)
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    result[np.where(mask == 255)] = warpedImage[np.where(mask == 255)]
    
    
    result = rgbEqualize(result)
    return result

In [9]:
def writeImages(images):
    for i in range(nImages):
        cv2.imwrite("log"+str(i)+".jpg",images[i])
        
        
writeImages(images)

In [10]:
def horizontalStitch(images):
    stitchOrder, matches = getOrdering(images)
    orderedImages = []
    for i in stitchOrder:
        orderedImages.append(images[i])
    centerImageIndex = len(orderedImages)//2
    print(centerImageIndex)
    
#     img = orderedImages[centerImageIndex]
#     for i in range(centerImageIndex+1,centerImageIndex+2):#len(orderedImages)):
#         img = warpTwoImages(img, orderedImages[i])
        
#     for i in range(centerImageIndex-1,centerImageIndex-2,-1):#len(orderedImages)):
#         img = warpTwoImages(img, orderedImages[i])
    
    
    rightStitchedImage = stitchFromRight(orderedImages[centerImageIndex], orderedImages[centerImageIndex+1:])
    leftStitchedImage = stitchFromLeft(rightStitchedImage, orderedImages[:centerImageIndex])
    
    return leftStitchedImage
#     return img

    
def stitchFromRight(image, images):
    if (len(images) == 0):
        return image
    warpedImg = warpTwoImages(image, images[0])
    return stitchFromRight(warpedImg, images[1:])
    
    
    
    
def stitchFromLeft(image, images):
    if (len(images) == 1):
        return image
    warpedImg = warpTwoImages(image, images[-1])
    return stitchFromLeft(warpedImg, images[:-1])
    
    
result = horizontalStitch(images)



0 1 0
0 2 0
0 3 1
0 4 0
0 5 0
0 6 2
0 7 2


1 2 0
1 3 0
1 4 6
1 5 0
1 6 0
1 7 1


2 3 0
2 4 0
2 5 0
2 6 0
2 7 1


3 4 0
3 5 0
3 6 1
3 7 0


4 5 0
4 6 1
4 7 0


5 6 1
5 7 1


6 7 0


[[0, 1], [2, 3], [4, 5], [6, 7]]
[[0, 1, 4, 5], [2, 3], [6, 7]]
[[2, 3], [6, 7, 0, 1, 4, 5]]
[1, 3, 0, 2]
2
shape of matches is: (7,)
shape of matches is: (2,)


error: OpenCV(4.1.1) /tmp/opencv-20190908-76835-1lnncvq/opencv-4.1.1/modules/core/src/matmul.dispatch.cpp:525: error: (-215:Assertion failed) scn + 1 == m.cols in function 'perspectiveTransform'


In [ ]:
cv2.imwrite("log.jpg",result)

In [ ]:
A = images[2]
B = images[1]

# generate Gaussian pyramid for A
G = A.copy()
gpA = [G]


for i in range(6):
    G = cv2.pyrDown(G)
    gpA.append(G)

# generate Gaussian pyramid for B
G = B.copy()
gpB = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpB.append(G)

# generate Laplacian Pyramid for A
lpA = [gpA[5]]
for i in range(5,0,-1):
    size = (gpA[i].shape[1], gpA[i].shape[0])
    GE = cv2.pyrUp(gpA[i], dstsize = size)
    print(type(gpB[i-1]), type(GE))
    L = np.subtract(gpA[i-1],GE)
    lpA.append(L)

# generate Laplacian Pyramid for B
lpB = [gpB[5]]
for i in range(5,0,-1):
    
    GE = cv2.pyrUp(gpB[i], dstsize = size)
    print(type(gpB[i-1]), type(GE))
    L = np.subtract(gpB[i-1],GE)
    
    lpB.append(L)

# Now add left and right halves of images in each level
LS = []
for la,lb in zip(lpA,lpB):
    rows,cols,dpt = la.shape
    ls = np.hstack((la[:,0:cols/2], lb[:,cols/2:]))
    LS.append(ls)

# now reconstruct
ls_ = LS[0]
for i in range(1,6):
    ls_ = cv2.pyrUp(ls_)
    ls_ = cv2.add(ls_, LS[i])

# image with direct connecting each half
real = np.hstack((A[:,:cols/2],B[:,cols/2:]))

cv2.imwrite('Pyramid_blending2.jpg',ls_)
cv2.imwrite('Direct_blending.jpg',real)